# nn.Module 继承 来构造模型

`Module`类是`nn`模块里提供的一个模型构造类，是所有神经网络模块的基类，我们可以继承它来定义我们想要的模型。下面继承`Module`类构造本节开头提到的多层感知机。这里定义的`MLP`类重载了`Module`类的`__init__`函数和`forward`函数。它们分别用于创建模型参数和定义前向计算。前向计算也即正向传播。

In [1]:
import torch
import torch.nn as nn

import sys
sys.path.append("..") 


class MLP(nn.Module):
    # 声明带有模型参数的层，这里声明了两个全连接层
    def __init__(self, **kwargs):
        # 调用MLP父类Block的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
        # 参数，如“模型参数的访问、初始化和共享”一节将介绍的模型参数params
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(28*28, 256)
        self.act = nn.ReLU()
        self.out = nn.Linear(256,10)
        
        # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
    def forward(self, x):
        x = self.act(self.hidden(x))
        return self.out(x)

In [2]:
x = torch.randn(2,1,28*28)
x = x.view(x.size()[0], -1)
net = MLP()
out = net(x)
print(out)


tensor([[ 0.0836, -0.0942,  0.2857, -0.1599, -0.0923, -0.3792, -0.5406, -0.1677,
          0.0389, -0.2380],
        [-0.1376,  0.1140, -0.1313,  0.1248,  0.0149, -0.3359, -0.4447, -0.0092,
         -0.2264, -0.1507]], grad_fn=<AddmmBackward>)


In [3]:
print(net)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (out): Linear(in_features=256, out_features=10, bias=True)
)


注意，这里并没有将`Module`类命名为`Layer`（层）或者`Model`（模型）之类的名字，这是因为该类是一个可供自由组建的部件。它的子类既可以是一个层（如PyTorch提供的`Linear`类），又可以是一个模型（如这里定义的`MLP`类），或者是模型的一个部分。我们下面通过两个例子来展示它的灵活性。

# 快速构建简单的网络，不需要写forward

## 都在nn工具箱下


### Squential 自动forward

1. Sequential 直接接受nn模块
>对于 Sequential的对象，可以 add_modules(继承Module的实例)

2. ModuleList 接受nn模块的list

> 方便像list 那样 append, extend

3. ModulDict  接受字典，方便命名，按名字访问，按名字添加属性，layer



In [4]:
from torch.nn import Sequential
net = Sequential(
        nn.Linear(784, 256),
        nn.ReLU(),
        nn.Linear(256, 10), 
        )
print(net)
out = net(x)
print(out)

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)
tensor([[-0.5808,  0.0198, -0.2878, -0.3476, -0.2464, -0.1148,  0.4248, -0.3322,
         -0.1394,  0.6659],
        [-0.5151,  0.1577, -0.0254, -0.1316,  0.3060, -0.0942,  0.3053, -0.3888,
          0.3985,  0.2867]], grad_fn=<AddmmBackward>)


In [5]:
net = nn.ModuleList([nn.Linear(784, 256),
        nn.ReLU(),
        nn.Linear(256, 10)])
print(net)
# 下面会报错,因为modellist没有实现forward方法
out = x
for layer in net:
    out = layer(out)
print(out)

net.append(nn.Linear(10,2))

print(net)

ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)
tensor([[ 0.1271,  0.3189,  0.4812, -0.2085, -0.0490,  0.4415, -0.0589,  0.3638,
         -0.6852,  0.1257],
        [-0.1920,  0.1337,  0.2488, -0.3550,  0.2886,  0.6090,  0.1322,  0.4318,
         -0.0815,  0.2190]], grad_fn=<AddmmBackward>)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
  (3): Linear(in_features=10, out_features=2, bias=True)
)


In [6]:
net = nn.ModuleDict({
    'linear':nn.Linear(28*28, 256),
    'act': nn.ReLU()
})
# 同样 module dict没有forward
#out = net(x)
print(net)
out = x

out = net['linear'](out)
print(out.shape)

#加入新的layer
net['layer2'] = nn.Linear(256,10)
print(net)

ModuleDict(
  (act): ReLU()
  (linear): Linear(in_features=784, out_features=256, bias=True)
)
torch.Size([2, 256])
ModuleDict(
  (act): ReLU()
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (layer2): Linear(in_features=256, out_features=10, bias=True)
)


## 构造复杂的模型

虽然上面介绍的这些类可以使模型构造更加简单，且不需要定义`forward`函数，但直接继承`Module`类可以极大地拓展模型构造的灵活性。下面我们构造一个稍微复杂点的网络`FancyMLP`。在这个网络中，我们通过`get_constant`函数创建训练中不被迭代的参数，即常数参数。在前向计算中，除了使用创建的常数参数外，我们还使用`Tensor`的函数和Python的控制流，并多次调用相同的层。


In [7]:
class FancyMLP(nn.Module):
    pass

In [8]:
class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential(nn.Linear(40, 30), nn.ReLU()) 

    def forward(self, x):
        return self.net(x)

In [9]:
#通过 nnModule 用nn.sequential 可以串联起来
net = nn.Sequential(NestMLP(), nn.Linear(30, 20), FancyMLP())

In [10]:
print(net)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP()
)


# 模型参数的初始化，访问、共享


一般来说，用nn工具箱中的Layer 都是会自动初始化的，给随机数等，nn.init模块有多种初始化方式

In [11]:
import torch
from torch import nn
from torch.nn import init

net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))  # pytorch已进行默认初始化

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


## 访问模型参数

回忆一下上一节中提到的`Sequential`类与`Module`类的继承关系。对于`Sequential`实例中含模型参数的层，我们可以通过`Module`类的`parameters()`或者`named_parameters`方法来访问所有参数（以迭代器的形式返回），后者除了返回参数`Tensor`外还会返回其名字。下面，访问多层感知机`net`的所有参数：

In [12]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name,'///' ,param.size())

<class 'generator'>
0.weight /// torch.Size([3, 4])
0.bias /// torch.Size([3])
2.weight /// torch.Size([1, 3])
2.bias /// torch.Size([1])


可见返回的名字自动加上了层数的索引作为前缀。
我们再来访问`net`中单层的参数。对于使用`Sequential`类构造的神经网络，我们可以通过方括号`[]`来访问网络的任一层。索引0表示隐藏层为`Sequential`实例最先添加的层。

In [13]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))
    print(param)

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([[ 0.4225,  0.4102, -0.4574, -0.2837],
        [ 0.0826,  0.2120, -0.2627,  0.3030],
        [ 0.1647,  0.1645, -0.1407, -0.4159]], requires_grad=True)
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.3954, -0.4933, -0.2842], requires_grad=True)


## 普通tensor 和 nn.Parameter
因为这里是单层的所以没有了层数索引的前缀。另外返回的`param`的类型为`torch.nn.parameter.Parameter`，其实这是`Tensor`的子类，和`Tensor`不同的是如果一个`Tensor`是`Parameter`，那么它会自动被添加到模型的参数列表里，来看下面这个例子。

In [14]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        # 只有 nn.Parameter 类型的tensor 才会加入到参数列表中，可以进行求导
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
    def forward(self, x):
        pass
    
n = MyModel()
for name, param in n.named_parameters():
    print(name, type(param))

weight1 <class 'torch.nn.parameter.Parameter'>


上面的代码中`weight1`在参数列表中但是`weight2`却没在参数列表中。

因为`Parameter`是`Tensor`，即`Tensor`拥有的属性它都有，比如可以根据`data`来访问参数数值，用`grad`来访问参数梯度。

## 初始化模型参数

我们在3.15节（数值稳定性和模型初始化）中提到了PyTorch中`nn.Module`的模块参数都采取了较为合理的初始化策略（不同类型的layer具体采样的哪一种初始化方法的可参考[源代码](https://github.com/pytorch/pytorch/tree/master/torch/nn/modules)）。但我们经常需要使用其他方法来初始化权重。PyTorch的`init`模块里提供了多种预设的初始化方法。在下面的例子中，我们将权重参数初始化成均值为0、标准差为0.01的正态分布随机数，并依然将偏差参数清零。

In [15]:
for name, param in net.named_parameters():
    if 'weight' in name:
        torch.nn.init.normal_(param,0,0.01)
        print(name, param)
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)
        

0.weight Parameter containing:
tensor([[ 0.0012,  0.0022, -0.0056, -0.0029],
        [-0.0029, -0.0102, -0.0020,  0.0151],
        [-0.0148, -0.0132,  0.0012,  0.0214]], requires_grad=True)
0.bias tensor([0., 0., 0.])
2.weight Parameter containing:
tensor([[0.0073, 0.0087, 0.0143]], requires_grad=True)
2.bias tensor([0.])


如果只想对某个特定参数进行初始化，我们可以调用`Parameter`类的`initialize`函数，它与`Block`类提供的`initialize`函数的使用方法一致。下例中我们对隐藏层的权重使用Xavier随机初始化方法。

##  自定义初始化方法

有时候我们需要的初始化方法并没有在`init`模块中提供。这时，可以实现一个初始化方法，从而能够像使用其他初始化方法那样使用它。

我们还可以通过改变这些参数的`data`来改写模型参数值同时不会影响梯度.


## 共享模型参数

在有些情况下，我们希望在多个层之间共享模型参数。4.1.3节提到了如何共享模型参数: `Module`类的`forward`函数里多次调用同一个层。此外，如果我们传入`Sequential`的模块是同一个`Module`实例的话参数也是共享的，下面来看一个例子: 

In [16]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear) 
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [17]:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))
#内存里其实是一个东西

True
True


In [23]:
x = torch.ones(1, 1)
print(x)
print(net(x))

y = net(x).sum()

print(y)


y.backward()

print(net[0].weight.grad) # 单次梯度是3，两次所以就是6

net[0].weight.grad.zero_() 

print(net[0].weight.grad) # 单次梯度是3，两次所以就是6
#weight.grad 是会累加的

tensor([[1.]])
tensor([[9.]], grad_fn=<MmBackward>)
tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
tensor([[0.]])



# 自定义层
 
## 如何使用自己定义的层呢

## 不含模型参数的自定义层

我们先介绍如何定义一个不含模型参数的自定义层。事实上，这和4.1节（模型构造）中介绍的使用`Module`类构造模型类似。下面的`CenteredLayer`类通过继承`Module`类自定义了一个将输入减掉均值后输出的层，并将层的计算定义在了`forward`函数里。这个层里不含模型参数。



我们可以实例化这个层，然后做前向计

In [24]:
import torch
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)
    def forward(self, x):
        return x - x.mean()

In [25]:
layer = CenteredLayer()
layer(torch.tensor([1, 2, 3, 4, 5], dtype=torch.float))

tensor([-2., -1.,  0.,  1.,  2.])

In [26]:
class MyDense(nn.Module):
    def __init__(self, **kwargs):
        super(MyDense, self).__init__()
        
        self.params = nn.ParameterList([nn.Parameter(torch.randn(4,4)) for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4,2)))
    def forward(self, x):
        for param in self.params:
            x = torch.mm(x, param)
        return x

net = MyDense()
print(net)

x = torch.randn(10,4)
out = net(x)
print(out)

MyDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 4x4]
      (1): Parameter containing: [torch.FloatTensor of size 4x4]
      (2): Parameter containing: [torch.FloatTensor of size 4x4]
      (3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)
tensor([[ 1.3558,  0.7263],
        [-3.9452,  0.8231],
        [ 0.0279,  9.6577],
        [ 3.6184,  7.3969],
        [-3.0278, -0.8494],
        [ 3.0580, -1.7761],
        [ 6.2205,  4.9560],
        [-2.5933, -8.6391],
        [ 2.7387,  2.3305],
        [-1.1401,  3.9867]], grad_fn=<MmBackward>)


而`ParameterDict`接收一个`Parameter`实例的字典作为输入然后得到一个参数字典，然后可以按照字典的规则使用了。例如使用`update()`新增参数，使用`keys()`返回所有键值，使用`items()`返回所有键值对等等，可参考

In [27]:
class MyDictDense(nn.Module):
    def __init__(self, **kwargs):
        super(MyDictDense, self).__init__()
        
        self.params = nn.ParameterDict({ 'layer1': nn.Parameter(torch.randn(4,4)),
                                       'layer2': nn.Parameter(torch.randn(4,1)),
                                       'layer3': nn.Parameter(torch.randn(1,4))})
        
        self.params.update({'out': nn.Parameter(torch.randn(4,2))})
    def forward(self, x):
        for layer_name in self.params:

            x = torch.mm(x, self.params[layer_name])
        return x

net = MyDictDense()
print(net)

x = torch.randn(10,4)
out = net(x)
print(out)

MyDictDense(
  (params): ParameterDict(
      (layer1): Parameter containing: [torch.FloatTensor of size 4x4]
      (layer2): Parameter containing: [torch.FloatTensor of size 4x1]
      (layer3): Parameter containing: [torch.FloatTensor of size 1x4]
      (out): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)
tensor([[-2.5305, -2.1880],
        [-0.4013, -0.3470],
        [-0.4136, -0.3576],
        [ 0.5182,  0.4480],
        [-0.5539, -0.4790],
        [-0.5887, -0.5090],
        [-0.7974, -0.6895],
        [-0.7974, -0.6895],
        [-0.5581, -0.4826],
        [-0.2467, -0.2133]], grad_fn=<MmBackward>)


# 同样可以加入sequential
可以通过`Module`类自定义神经网络中的层，从而可以被重复调用。

In [28]:
net = nn.Sequential(
    MyDictDense(),
    nn.Linear(2,4),
    MyDense(),
)

print(net)
print(net(x))

Sequential(
  (0): MyDictDense(
    (params): ParameterDict(
        (layer1): Parameter containing: [torch.FloatTensor of size 4x4]
        (layer2): Parameter containing: [torch.FloatTensor of size 4x1]
        (layer3): Parameter containing: [torch.FloatTensor of size 1x4]
        (out): Parameter containing: [torch.FloatTensor of size 4x2]
    )
  )
  (1): Linear(in_features=2, out_features=4, bias=True)
  (2): MyDense(
    (params): ParameterList(
        (0): Parameter containing: [torch.FloatTensor of size 4x4]
        (1): Parameter containing: [torch.FloatTensor of size 4x4]
        (2): Parameter containing: [torch.FloatTensor of size 4x4]
        (3): Parameter containing: [torch.FloatTensor of size 4x2]
    )
  )
)
tensor([[-423.7537, -281.0964],
        [-362.3016, -239.7750],
        [  48.4405,   36.4148],
        [-399.2125, -264.5945],
        [  70.8420,   51.4779],
        [  61.0302,   44.8803],
        [-131.3318,  -84.4671],
        [-190.2179, -124.0631],
       